In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook

# ================= SCRAPE =================
url = "https://sarmaaya.pk/indexes/KMI30"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get(url)

WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr"))
)

html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, "html.parser")
rows = soup.select("tbody tr")

data = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 4:
        continue

    data.append({
        "Stock": cols[0].get_text(strip=True),
        "Weight (%)": float(cols[2].get_text(strip=True)),
        "Price": float(cols[3].get_text(strip=True))
    })

df = pd.DataFrame(data)

# ================= SORT BY WEIGHT =================
df = df.sort_values("Weight (%)", ascending=False).reset_index(drop=True)

# ================= WRITE EXCEL =================
file_path = "KMI30_Monthly_Investment.xlsx"
df.to_excel(file_path, index=False, startrow=4, header=False)

wb = load_workbook(file_path)
ws = wb.active

# User input
ws["A1"] = "Total Monthly Investment"
ws["B1"] = 5000   # USER CHANGES THIS

# Headers
ws["A3"] = "Stock"
ws["B3"] = "Weight (%)"
ws["C3"] = "Price"
ws["D3"] = "Allocated Amount"
ws["E3"] = "Shares to Buy"

start_row = 5
end_row = start_row + len(df) - 1

for r in range(start_row, end_row + 1):
    ws[f"D{r}"] = f"=$B$1*B{r}/100"
    ws[f"E{r}"] = f"=D{r}/C{r}"

wb.save(file_path)

print("Excel file created:", file_path)


Excel file created: KMI30_Monthly_Investment.xlsx


In [7]:
import yfinance as yf
import plotly.graph_objects as go

def get_stock_data(symbol, period="1y"):
    ticker = f"{symbol}.KA"
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

def plot_line_chart(df, symbol):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Close Price'
    ))

    fig.update_layout(
        title=f'{symbol} Price History',
        xaxis_title='Date',
        yaxis_title='Price (PKR)'
    )
    fig.show()

# Example Usage
df = get_stock_data("LUCK")
plot_line_chart(df, "Systems Ltd")


In [4]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2025-01-30 00:00:00+05:00,575.609991,588.578673,562.641308,585.974976,177702,0.0,0.0,0.0
2025-01-31 00:00:00+05:00,587.451441,598.554635,582.593178,593.427002,232949,0.0,0.0,0.0
2025-02-03 00:00:00+05:00,593.426972,593.426972,573.614829,575.570129,46896,0.0,0.0,0.0
2025-02-04 00:00:00+05:00,580.777501,583.590715,573.634777,580.059265,27203,0.0,0.0,0.0
2025-02-06 00:00:00+05:00,574.861853,582.593184,553.663043,557.992615,331756,0.0,0.0,0.0
